# Energy Participation

In [4]:
####################
##   PARAMETERS   ##
####################

import numpy as np

#Constants used for calculations
e = 1.60218e-19
hbar = 1.05457e-34
phi0 = hbar/(2*e)

#Josephson Energy (Ej) and linear inductance (Lj)

delta_super = (1.8e-4 / 2) * e
#Ej = (0.5 * delta_super * hbar * 2 * np.pi) / ((2*e)**2 * 4444) #value for designed Ej = 8.41 GHz Rn = 8345, Divita test = 4444 Ohm
#Ej_ghz = 15.8251e9
#Ej = 2 * np.pi * hbar * Ej_ghz 
#Lj = phi0**2 / Ej
Lj = 14e-9 
Ej = phi0**2 / Lj

#Enter in Participation Ratio from Simulation
P_qubit = 0.9926 #0.9926
P_cav = 0.0031 #0.002287

#Qiskit_EPR for transmon and resonator P_qubit = 0.9915, P_resonator = 0.002273, qubit_freq = 6.22071e9, res_freq = 9.4382e9, Lj = 10.00e-9
#Qiskit_EPR for transmon and resonator (Strong meshing/higher order) P_qubit = 0.99088, P_resonator = 0.0024975, qubit_freq = 6.45729e9, res_freq = 9.50827e9, Lj = 10.00e-9
#Divita Device Q4 P_qubit = 0.9926, P_resonator = 0.002287, qubit_freq = 6.0714e9, res_freq = 7.5651e9, Lj = 10.345e-9 

#Normalize Participation Ratios
P_qubit_normalised = P_qubit / (P_qubit + P_cav)
P_cav_normalised = P_cav / (P_qubit + P_cav)

#Enter in Frequencies from Simulation
linear_qubit_freq = 6.358e9
linear_res_freq = 9.484e9
omega_qubit = 2 * np.pi * linear_qubit_freq
omega_res = 2 * np.pi * linear_res_freq

####################
##  CALCULATIONS  ##
####################

#phi zero-point fluctuations for qubit
phi_zpf_sq = P_qubit * hbar * 2 * omega_qubit  / (2*Ej)

#anharmonicity qubit (self kerr)
anharm_qubit = P_qubit**2 * hbar * omega_qubit**2 / (8*Ej)

#anharmonicity resonator (self kerr)
anharm_res = P_cav**2 * hbar * omega_res**2 / (8*Ej)

#Charging Energy (Ec)
Ec = anharm_qubit * hbar

#Total Capacitance
C_total = e**2 / (2*Ec)

#cross kerr - dispersive shit (chi)
cross_kerr = (P_qubit * P_cav * hbar * omega_qubit * omega_res) / (4*Ej)

#lamb-shift of qubit frequency
lamb_shift_qubit = anharm_qubit - cross_kerr/2

#lamb-shift of res frequency
lamb_shift_res = anharm_res - cross_kerr/2

#qubit frequency using standard formula (check if calculated frequency makes sense)
qubit_freq_test = (np.sqrt(8*Ej * Ec) - Ec)/ hbar

#detuning, delta
delta = ((linear_res_freq - anharm_res/(2*np.pi)) - (linear_qubit_freq - anharm_qubit/(2*np.pi)))

#resonator-qubit coupling strength (g)
g = np.sqrt(disp_shift_qubit * delta  * (1 + delta/anharm_qubit))

#for comparison with cross kerr (Krantz et al. (2021) p42)
chi_test = (g**2 / delta) * (1 / (1 + delta/(anharm_qubit)))

#test for Ec/Anharmonicity using omega_qubit = sqrt(8 * Ec * Ej)/hbar
Ec_test = (omega_qubit * hbar)**2 / (8*Ej)
anharm_qubit_test = Ec_test/(hbar*2*np.pi*1e6)

#critical photon number
nc = delta**2 / (4*g**2)

#print values
print('--------------')
print('|Calculations|')
print('--------------')
print('Ej:\t\t\t\t\t', '%.4f' % (Ej / (hbar * 2 * np.pi *  1e9)), 'GHz', '\t(' '%.4e' %  (Ej ), 'J)')
print('Ec:\t\t\t\t\t', '%.4f' % (anharm_qubit  / (2*np.pi*1e9)), 'GHz', '\t(' '%.4e' %  (anharm_qubit*hbar), 'J)' )
print('Lj:\t\t\t\t\t', '%.4f' %  (Lj/1e-9), 'nH')
print('Cj:\t\t\t\t\t', '%.4f' %  (C_total/1e-15), 'fF')
print('Participation Ratio Qubit:\t\t', '%.5f' % P_qubit, '\t(Normalized =',  '%.5f' % P_qubit_normalised,')')
print('Participation Ratio Cavity:\t\t', '%.5f' % P_cav, '\t(Normalized =',  '%.5f' % P_cav_normalised,')')
print('Linear Qubit Frequency:\t\t\t', '%.4f' % (linear_qubit_freq/1e9), 'GHz')
print('Linear Res Frequency:\t\t\t', '%.4f' % (linear_res_freq/1e9), 'GHz')
print('Qubit Anharmonicity (alpha_Q):\t\t', '%.4f' % (anharm_qubit / (2*np.pi*1e6)), 'MHz')
print('Res Anharmonicity (alpha_R):\t\t', '%.4f' % (anharm_res / (2*np.pi*1e6)), 'MHz')
print('Dispersive Shift (chi):\t\t\t', '%.4f' % (cross_kerr/ (2*np.pi*1e6)), 'MHz')
print('Qubit Frequency:\t\t\t', '%.4f' % (linear_qubit_freq/1e9 - lamb_shift_qubit/(2*np.pi*1e9)), 'GHz')
print('Res Frequency:\t\t\t\t', '%.4f' % (linear_res_freq/1e9 - lamb_shift_res/(2*np.pi*1e9)), 'GHz')
print('Res-Qubit Coupling (g):\t\t\t', '%.4f' % (g/(2*np.pi*1e6)), 'MHz')
print('Detuning:\t\t\t\t', '%.4f' % (delta/1e9), 'GHz')
print('Ej/Ec:\t\t\t\t\t', '%.4f' % (Ej/Ec))
print('Flux_ZPF_squared:\t\t\t', '%.4f' % (phi_zpf_sq))
print('Critical Photon Number:\t\t\t', '%.4f' % (nc))


NameError: name 'disp_shift_qubit' is not defined

In [6]:
1.0830991660730458e-23/hbar /1e9

102.7052889872693

In [7]:
Ej/hbar / 1e9

73.36092070519237

In [65]:
alpha = (0.5 * Ej * phi_zpf_sq)/hbar
print(alpha/(2*np.pi*1e6))
print(chi_test/(2*np.pi*1e6))
hbar/(2*e)
E_charge = (omega_qubit * hbar)**2 / (8*Ej)
E_charge/(hbar*2*np.pi*1e6)


3009.896550000001
2.6403569357204875


291.61191173768555

In [126]:
np.sqrt(83.39**2 + 46.17**2)
np.degrees(np.arctan(46.17/83.39))
e**2 / (2*67.5224e-15) / (hbar*2*np.pi*1e9)


0.28687286146128693

In [120]:
e**2 / (61.4672e-15) / (hbar*2*np.pi*1e9)


0.6302660313381315

In [112]:
EJ = 15.825e9 * 2*np.pi* hbar
EC = 286.9e6 * 2*np.pi* hbar #315e6 designed
qubit_frequency = ((np.sqrt(8*EJ*EC) - EC) / hbar) / (2*np.pi*1e9)
print(qubit_frequency, 'GHz')

5.739835434710902 GHz


In [123]:
EJ/EC
11431/60

190.51666666666668

In [130]:
(8.03 - 7.97) / 501

0.0001197604790419154

In [21]:
import numpy as np
e = 1.60218e-19
hbar = 1.05457e-34
phi0 = hbar/(2*e)
Lj = phi0/1e-8
Ej = phi0**2 / Lj

RN = hbar*2*np.pi* (180e-6*e) / (2 * Ej * 2*e**2)
RN

56548.66776461628

In [19]:
Lj = phi0/1e-7
Lj

3.2910471981924628e-09

In [1]:
8997/60

149.95